https://cloud.google.com/ml-engine/docs/command-line
Create a project and enable it


apt-get install sudo curl

In [ ]:
https://cloud.google.com/sdk/docs/quickstart-debian-ubuntu
# Create an environment variable for the correct distribution
export CLOUD_SDK_REPO="cloud-sdk-$(lsb_release -c -s)"

# Add the Cloud SDK distribution URI as a package source
echo "deb http://packages.cloud.google.com/apt $CLOUD_SDK_REPO main" | sudo tee -a /etc/apt/sources.list.d/google-cloud-sdk.list

# Import the Google Cloud Platform public key
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

# Update the package list and install the Cloud SDK
sudo apt-get update && sudo apt-get install google-cloud-sdk

In [ ]:
gcloud init
gcloud config set project ai-ml-dl


In [ ]:
https://cloud.google.com/ml-engine/docs/getting-started-training-prediction

Export ml,  Model validation failed: Serving metagraph must contain exactly one SignatureDef with key: serving_default

In [1]:
import sys
sys.path.insert(0, '../aux/')
from beiras_aux import load_coded_dictionaries, predict_next_chars, clean_text
from keras.layers import Dense, Activation, GRU
from keras.models import Sequential
from keras.layers import InputLayer
import os

# Defining the model
def create_gru_model( num_chars):
    """
    Define the network
    :param
        numbers_chars .- Number chars using in the training process
    :return:
        model .- Model network defined
    """
    model = Sequential()
    # 1 Layer .- GRU layer 1 should be an GRU module with 200 hidden units
    model.add(GRU(200, input_shape=(window_size, num_chars), return_sequences=True))
    # 2 Layer .- GRU layer 2 should be an GRU module with 200 hidden units
    model.add(GRU(200))
    # 2 Layer .-  Dense, with number chars unit and softmax activation
    model.add(Dense(num_chars, activation='softmax'))
    return model

import keras.backend as K
K.set_learning_phase(0)

# Input size of the network, the entry text must have the same length
window_size = 100
# Get dictionaries
chars_to_indices, indices_to_chars = load_coded_dictionaries()
number_chars=len(chars_to_indices)
# regenerate the model
model=create_gru_model(number_chars)
model.load_weights('../model_weights/best_beiras_gru_textdata_weights.hdf5')


from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model.signature_def_utils_impl import build_signature_def, predict_signature_def
from tensorflow.contrib.session_bundle import exporter

# Path to export, 1 is the version, 
# we can serve differents version with the same server
export_path = "../export-google-ml/1"
import shutil
import os


if os.path.isdir(export_path):
    shutil.rmtree(export_path)
builder = saved_model_builder.SavedModelBuilder(export_path)

signature = predict_signature_def(inputs={'sequence': model.input},
                                  outputs={'scores': model.output})

with K.get_session() as sess:
    builder.add_meta_graph_and_variables(sess=sess,
                                         tags=[tag_constants.SERVING],
                                         signature_def_map={'serving_default': signature})
    builder.save()


Using TensorFlow backend.
/home/jota/anaconda3/envs/beiras-rnn/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'../export-google-ml/1/saved_model.pb'


In [ ]:
Make bucket an upload
gsutil mb gs://beiras_rnn/
gsutil cp -r ./export-google-ml/ gs://beiras_rnn/


In [ ]:
MODEL_NAME=BeirasRnn
gcloud ml-engine models create $MODEL_NAME --enable-logging
DEPLOYMENT_SOURCE="gs://beiras_rnn/export-google-ml/1"
gcloud ml-engine versions create "v02" --model $MODEL_NAME  --origin $DEPLOYMENT_SOURCE


In [71]:
import sys
import numpy as np
sys.path.insert(0, '../aux/')
from beiras_aux import load_coded_dictionaries, predict_next_chars, clean_text


input_init="se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu"
input_init="pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran"
# Load values
window_size = 100
chars_to_indices, indices_to_chars = load_coded_dictionaries()
number_chars=len(chars_to_indices)
# Clean the text
input_clean=clean_text(input_init.lower())
input_clean = input_clean[:window_size]
# Text to array [1,input_lenght,num_chars]
#x_test = np.zeros((1,window_size, number_chars))
#for t, char in enumerate(input_clean):
#    x_test[0, t, chars_to_indices[char]] = 1.
#x_test   

## Ollo, hai que invertir frente o que empregar normalmente filas x columnas
x_test = np.zeros((number_chars,window_size))
for t, char in enumerate(input_clean):
    x_test[ chars_to_indices[char],t] = 1.

In [ ]:
pip install --upgrade google-api-python-client

In [ ]:
cloud auth application-default login,

In [109]:
import googleapiclient.discovery
project="ai-ml-dl"
model="BeirasRnn"
version=None
service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}'.format(project, model)
if version is not None:
        name += '/versions/{}'.format(version)
instances={'sequence':x_test.tolist()}
response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()
if 'error' in response:
    raise RuntimeError(response['error'])
test_predict=np.array(response['predictions'][0]['scores'])
r = np.argmax(test_predict)  # predict class of each test input
indices_to_chars[r]


[0.15315677225589752, 5.445067140230631e-13, 0.00888405553996563, 3.2357725388010294e-08, 1.7484167983639054e-05, 6.10320275882259e-05, 1.2296323802729603e-05, 1.1282341802143492e-05, 3.179920167895034e-05, 3.8525176933035254e-05, 2.2138216081657447e-05, 2.391875023022294e-05, 8.11919198895339e-06, 0.0001562454126542434, 0.024670373648405075, 0.0025260080583393574, 0.001229702727869153, 0.0003667160344775766, 0.15842320024967194, 0.000622001476585865, 0.012139963917434216, 0.003239764366298914, 0.08750336617231369, 5.42994439456379e-06, 7.810050192347262e-06, 0.007188642863184214, 0.005238964222371578, 0.009875708259642124, 0.006339691113680601, 0.0020037461072206497, 0.0003367718309164047, 0.0021479781717061996, 0.5080389380455017, 0.0013852526899427176, 0.0022390142548829317, 0.001438260544091463, 4.310204076318769e-06, 4.12502049584873e-05, 2.2399261069949716e-05, 1.0920505701506045e-05, 5.618276568801484e-13, 4.796372717399966e-13, 9.346359729534015e-05, 6.586684865063364e-13, 0.00

's'

Predict from console

In [69]:
import json
with open('data.json', 'w') as fp:
    json.dump(instances, fp)

In [ ]:
gcloud ml-engine predict --model=BeirasRnn --json-instances=data.json


1

Predict with funcions


In [126]:
# Define a function
def predict_one(text_predict,service,model_name,window_size,chars_to_indices, indices_to_chars):
    # Convert input sequence to array
    number_chars=len(chars_to_indices)
    x_test = np.zeros((number_chars,window_size))
    for t, char in enumerate(text_predict):
        x_test[chars_to_indices[char],t] = 1.
    #Prepare the request
    instances={'sequence':x_test.tolist()}
    response = service.projects().predict(
        name=name,
        body={'instances': instances}
    ).execute()
    if 'error' in response:
        raise RuntimeError(response['error'])
    test_predict=np.array(response['predictions'][0]['scores'])
    r = np.argmax(test_predict)  # predict class of each test input
    print(text_predict)
    print(test_predict)
    print(r,indices_to_chars[r])
    return (indices_to_chars[r])

In [127]:
project="ai-ml-dl"
model="BeirasRnn"
version=None

# Complete a sequence using the server
def predict_window(text_predict,number_predict,window_size):
    
    # Get dictionaries
    chars_to_indices, indices_to_chars = load_coded_dictionaries()
    # Clean the test
    input_clean=clean_text(text_predict.lower())
    # Get stub
    service = googleapiclient.discovery.build('ml', 'v1')
    name = 'projects/{}/models/{}'.format(project, model)
    if version is not None:
        name += '/versions/{}'.format(version)
    # Call server for all charazters
    for i in range(number_predict):
        d=predict_one(input_clean[i:],service,name,window_size,chars_to_indices, indices_to_chars)
        input_clean+=d
    return input_clean

In [128]:
import googleapiclient.discovery


input_init="se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu"
#input_init="pla panfletaria contra as leoninas taxas impostas polo ministro de xustiza actual malia que vulneran"

window_size = 100

       

response=predict_window(input_init[:window_size ],10,window_size)
response

se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcu
[  3.02791549e-03   1.82074245e-13   1.66169353e-04   7.92613406e-08
   5.73130601e-05   1.31240768e-05   2.25630544e-07   1.25827592e-05
   3.36391054e-06   2.49683217e-06   2.67709061e-06   2.58707291e-06
   3.85522708e-06   1.68106453e-06   9.78884920e-02   7.72795000e-04
   1.28873562e-05   6.51064329e-05   4.04933803e-02   5.68993273e-05
   7.88223770e-05   2.52500555e-04   7.03036606e-01   2.65130211e-06
   2.58586806e-05   6.67042732e-02   1.81544849e-04   4.20611771e-03
   6.22344902e-03   1.42863850e-04   1.35668085e-06   8.01028404e-03
   6.67816699e-02   1.40305041e-04   2.27522425e-04   2.22166782e-05
   3.05614267e-05   1.81408183e-07   9.27245637e-05   1.32501589e-06
   1.63243293e-13   1.66929716e-13   1.00698369e-03   2.05680620e-13
   6.43400635e-05   1.30786441e-13   1.48362204e-04   1.64503529e-13
   2.37581412e-06   1.64584519e-13   1.40918200e-05   1.60686067e-13
  

 cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcui  ie   
[  3.61237489e-02   6.55861060e-11   8.52909591e-03   5.70533030e-06
   1.51034910e-03   7.21118879e-04   7.27899387e-05   1.12317994e-05
   1.11934960e-05   1.72465716e-05   1.24900698e-05   2.90920229e-06
   1.96032015e-05   8.18335684e-04   1.25901341e-01   2.64307912e-02
   1.25886379e-02   7.42209551e-04   5.07524669e-01   1.04188242e-04
   1.48752276e-02   4.03802209e-02   1.83462389e-02   1.50350406e-05
   2.44835628e-05   5.50457202e-02   1.83839891e-02   8.83430988e-03
   8.45273659e-02   7.75562041e-03   8.20824716e-05   1.40286551e-03
   1.51101472e-02   1.11502397e-03   4.70428634e-03   8.46202718e-04
   1.40122953e-04   8.65625680e-06   7.05360289e-05   3.72911086e-06
   7.10649664e-11   6.69536995e-11   3.06642894e-03   6.98203023e-11
   2.81504053e-03   5.75108849e-11   2.55503925e-04   6.96157854e-11
   1.92159023e-05   7.49013976e-11   5.69736469e-04   7.04777903e-11
  

'se moito cando dixen eu que as suas políticas agresoras do común cidadán matan e a sua cospedal alcui  ie   ei'